###Projeto Machine Learning II

Usamos o dataset Stroke Prediction Dataset e vamos aplicar o Métodos SVM, AdaBoos e XGBoost

Link para o dataset utilizado: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset


In [33]:
#Importando as bibs importantes
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [34]:
# Carregar o dataset (depois de ter um conhecimento prévio dele)
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [35]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [36]:
# Análise exporatória de dados
# nulos (missing), dados duplicados, distribuição, (tratamento de dados), dados categóricos/numéricos
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [37]:
df.describe(include="object")

,gender,ever_married,work_type,Residence_type,smoking_status
count,5110,5110,5110,5110,5110
unique,3,2,5,2,4
top,Female,Yes,Private,Urban,never smoked
freq,2994,3353,2925,2596,1892


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [39]:
df.drop(columns="id")

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [40]:
#arredondando a idade para fazer mais sentido

df = df.drop(columns = 'id')
df['age'] = df['age'].apply(lambda x : round(x))
#substituindo os bmi's maiores que 50 e menores que 15 por não haver muitas
# na prática pessoas com estes valores de IMC
df['bmi'] = df['bmi'].apply(lambda bmi_value: bmi_value if 15 <
                          bmi_value < 50 else np.nan)
#preenchendo o índice de massa corporal pela média da idade da entrada
#para aqueles casos em que temos NaN
mean_values = df.groupby('age')['bmi'].transform('mean')
df['bmi'].fillna(mean_values, inplace=True)
#transformando as colunas object em colunas int64 para ver a correlaçao
encoded_cols = df.select_dtypes(exclude=['number']).columns.tolist()
label_encoder = LabelEncoder()
for col in encoded_cols:
    df[col] = label_encoder.fit_transform(df[col])




In [41]:
#testando se temos ainda alguma valor nulo
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67,0,1,1,2,1,228.69,36.600000,1,1
1,0,61,0,0,1,3,0,202.21,29.983824,2,1
2,1,80,0,1,1,2,0,105.92,32.500000,2,1
3,0,49,0,0,1,2,1,171.23,34.400000,3,1
4,0,79,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,0,80,1,0,1,2,1,83.75,28.783582,2,0
5106,0,81,0,0,1,3,1,125.20,40.000000,2,0
5107,0,35,0,0,1,3,0,82.99,30.600000,2,0
5108,1,51,0,0,1,2,0,166.29,25.600000,1,0


In [42]:
encoded_cols

['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']